<a href="https://colab.research.google.com/github/patmakur27/ptmakur27/blob/main/crewai_26th.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

  Using cached typer-0.9.4-py3-none-any.whl.metadata (14 kB)
Using cached typer-0.9.4-py3-none-any.whl (45 kB)
  Attempting uninstall: typer
    Found existing installation: typer 0.15.3
    Uninstalling typer-0.15.3:
      Successfully uninstalled typer-0.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.29.0 requires typer<1.0,>=0.12; sys_platform != "emscripten", but you have typer 0.9.4 which is incompatible.


In [ ]:
pip install gradio

  Using cached typer-0.15.3-py3-none-any.whl.metadata (15 kB)
Using cached typer-0.15.3-py3-none-any.whl (45 kB)
  Attempting uninstall: typer
    Found existing installation: typer 0.9.4
    Uninstalling typer-0.9.4:
      Successfully uninstalled typer-0.9.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
instructor 0.5.2 requires typer<0.10.0,>=0.9.0, but you have typer 0.15.3 which is incompatible.


In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [ ]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [ ]:
import os

os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [ ]:
import os
from google.colab import userdata

# Retrieve API key from Colab secrets and set it in environment variables
os.environ["OPENAI_API_KEY"] = userdata.get('openai')

In [ ]:
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)

In [ ]:
# Set up API keys - enter from https://serper.dev/billing
os.environ["SERPER_API_KEY"] = "" # serper.dev API key

# Instantiate tools
docs_tool = DirectoryReadTool(directory='./blog-posts')
file_tool = FileReadTool()
search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()

In [ ]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic."    ,
    tools=[search_tool, web_rag_tool,docs_tool,file_tool],
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [ ]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic in 1500 words: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [ ]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [ ]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [ ]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.1500 words",
    agent=writer,
)

### Task: Edit

In [ ]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution.

In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    planning=True,
    verbose=4
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [ ]:
result = crew.kickoff(inputs={"topic": "Model Context Protocol"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Model Context Protocol.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I should start by researching the latest trends, key players, and news on Model Context Protocol to prioritize the information. Then, I will identify the target audience and their interests and pain points to tailor the content to their needs. Next, I will develop a detailed outline with an introduction, key points, and a call to action, while also including relevant SEO keywords and credible sources.

Action: Search the internet
Action Input: {"search_query": "latest trends in Model Context Protocol"} 

{'message': 'Unauthorized. Sign up for a free accou

- Display the results of your execution as markdown in the notebook.

In [ ]:
from IPython.display import Markdown
Markdown(result)

# Exploring the Model Context Protocol: Trends, Players, and Insights

## Introduction
The Model Context Protocol is a critical component of data management and integration within the tech industry. Staying informed about the latest trends and key players in this field is essential for professionals seeking to stay ahead in a rapidly evolving landscape. By diving into current developments and advancements, valuable insights can be gained into the impact of the Model Context Protocol on the industry and its potential future implications.

## Latest Trends in Model Context Protocol
Recent years have witnessed significant advancements and innovations in the Model Context Protocol. From enhanced data integration capabilities to improved interoperability, these trends are reshaping how organizations handle their data. The adoption of new technologies and protocols is fostering efficiency and productivity in data management processes, ultimately leading to a more streamlined and integrated approach.

## Key Players in Model Context Protocol
Major companies and organizations play a crucial role in shaping the Model Context Protocol landscape. By profiling these key players and exploring their contributions to the field, a better understanding of their impact and influence can be gained. Collaborations between industry giants and innovative startups are propelling progress and pushing the boundaries of what is achievable in data integration.

## Noteworthy News in Model Context Protocol
Remaining up-to-date on the latest news, events, and announcements in the Model Context Protocol space is imperative for professionals looking to stay informed. Analyzing the significance and relevance of recent updates provides valuable insights into the industry's direction and potential opportunities for growth and development.

## Target Audience Analysis
Professionals in the tech industry, particularly those involved in data management and integration, are the primary audience for this blog article. Their interest in emerging technologies and data protocols makes them ideal candidates for exploring trends and insights in the Model Context Protocol. Addressing their pain points related to data management and integration will offer valuable information and solutions to help navigate the industry's complexities.

## Content Tailoring
Customizing the content to meet the needs of the target audience is crucial for engaging and informing readers. By incorporating real-world examples and case studies, practical insights and actionable takeaways can be provided that resonate with tech industry professionals. This tailored approach ensures that the content is relevant and valuable to the intended audience.

## Conclusion
In conclusion, the Model Context Protocol presents exciting opportunities for professionals in the tech industry as a dynamic and evolving field. By examining the latest trends, key players, and noteworthy news in this space, valuable insights into the impact of data integration and technology advancements on the industry can be gained. Staying informed and engaged with the latest developments will be crucial for success as we continue to push the boundaries of what is feasible in data management and integration.

## Call to Action
Readers are encouraged to delve into further reading and research on the Model Context Protocol to deepen their understanding of this significant topic. By staying informed and engaged with emerging technologies and data protocols, professionals can position themselves for success in an ever-changing industry. Thoughts and feedback on this blog article are welcomed, along with continued exploration of related topics to remain informed and up-to-date on the latest tech industry trends.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "AI Legislation"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on AI Legislation.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I need to gather information on the latest trends, key players, and noteworthy news on AI Legislation to create a comprehensive content plan.

Action: Search the internet
Action Input: {"search_query": "latest trends in AI Legislation"} 

{'message': 'Unauthorized. Sign up for a free account.', 'statusCode': 403}

Final Answer: Due to access restrictions, I was unable to gather information on the latest trends in AI Legislation through a simple internet search.

> Finished chain.
 [DEBUG]: == [Content Planner] Task output: Due to access restrictions, I was unable

In [ ]:
from IPython.display import Markdown
Markdown(result)

# The Impact of AI Legislation on Society

Artificial Intelligence (AI) has become an integral part of our daily lives, from virtual assistants like Siri to self-driving cars and predictive algorithms in healthcare. As AI technology continues to advance at a rapid pace, there is a growing concern about the need for proper legislation to govern its use. The question of how to regulate AI has become a hot topic of debate among policymakers, industry leaders, and ethicists alike. In this blog post, we will explore the implications of AI legislation on society and the challenges that come with regulating this rapidly evolving technology.

## The Need for AI Legislation

AI has the potential to revolutionize industries and improve efficiency in various sectors. However, the lack of clear regulations governing its use raises concerns about potential negative consequences. Without proper oversight, AI systems can perpetuate bias, invade privacy, and even pose risks to national security. For example, facial recognition technology has been criticized for its lack of accuracy, especially when it comes to identifying individuals of certain races or ethnicities. In the absence of regulations, there is a risk that AI systems could be used in ways that harm rather than benefit society.

## Challenges in Regulating AI

One of the main challenges in regulating AI is the speed at which the technology is advancing. Traditional legislative processes are often slow and cumbersome, making it difficult to keep up with the rapid pace of innovation in the AI industry. Additionally, the complexity of AI systems makes it challenging to create regulations that are both effective and flexible enough to accommodate future developments. Another challenge is the lack of consensus on what constitutes ethical AI. Different stakeholders may have conflicting views on the ethical implications of AI technology, making it challenging to create regulations that satisfy all parties involved.

## The Role of Government in AI Legislation

Governments play a crucial role in shaping AI legislation and ensuring that AI technology is used responsibly. By enacting laws and regulations that govern the development and deployment of AI systems, governments can help mitigate potential risks and ensure that AI is used in a way that benefits society as a whole. For example, the European Union recently introduced the General Data Protection Regulation (GDPR), which includes provisions on the use of AI and data protection. The GDPR aims to protect individuals' privacy rights while also promoting innovation in AI technology.

## Conclusion

In conclusion, AI legislation is a complex and multifaceted issue that requires careful consideration and collaboration among various stakeholders. As AI technology continues to evolve, it is crucial that policymakers work together to create regulations that promote the responsible use of AI while also fostering innovation and economic growth. By addressing the challenges associated with regulating AI and taking into account the ethical implications of this technology, we can ensure that AI benefits society as a whole. It is imperative that we stay ahead of the curve and proactively address the challenges posed by AI through thoughtful and comprehensive legislation. 

In the ever-evolving landscape of AI technology, legislation will play a critical role in shaping the future of society. It is up to us to ensure that AI is harnessed for the greater good and that regulations are in place to protect individuals and communities from potential harm. As we navigate the complexities of AI legislation, let us strive to strike a balance between innovation and ethical considerations, setting the stage for a future where AI technology can truly benefit society.

In [ ]:
Markdown(result)

# The Impact of AI Legislation on Society

Artificial Intelligence (AI) has become an integral part of our daily lives, from virtual assistants like Siri to self-driving cars and predictive algorithms in healthcare. As AI technology continues to advance at a rapid pace, there is a growing concern about the need for proper legislation to govern its use. The question of how to regulate AI has become a hot topic of debate among policymakers, industry leaders, and ethicists alike. In this blog post, we will explore the implications of AI legislation on society and the challenges that come with regulating this rapidly evolving technology.

## The Need for AI Legislation

AI has the potential to revolutionize industries and improve efficiency in various sectors. However, the lack of clear regulations governing its use raises concerns about potential negative consequences. Without proper oversight, AI systems can perpetuate bias, invade privacy, and even pose risks to national security. For example, facial recognition technology has been criticized for its lack of accuracy, especially when it comes to identifying individuals of certain races or ethnicities. In the absence of regulations, there is a risk that AI systems could be used in ways that harm rather than benefit society.

## Challenges in Regulating AI

One of the main challenges in regulating AI is the speed at which the technology is advancing. Traditional legislative processes are often slow and cumbersome, making it difficult to keep up with the rapid pace of innovation in the AI industry. Additionally, the complexity of AI systems makes it challenging to create regulations that are both effective and flexible enough to accommodate future developments. Another challenge is the lack of consensus on what constitutes ethical AI. Different stakeholders may have conflicting views on the ethical implications of AI technology, making it challenging to create regulations that satisfy all parties involved.

## The Role of Government in AI Legislation

Governments play a crucial role in shaping AI legislation and ensuring that AI technology is used responsibly. By enacting laws and regulations that govern the development and deployment of AI systems, governments can help mitigate potential risks and ensure that AI is used in a way that benefits society as a whole. For example, the European Union recently introduced the General Data Protection Regulation (GDPR), which includes provisions on the use of AI and data protection. The GDPR aims to protect individuals' privacy rights while also promoting innovation in AI technology.

## Conclusion

In conclusion, AI legislation is a complex and multifaceted issue that requires careful consideration and collaboration among various stakeholders. As AI technology continues to evolve, it is crucial that policymakers work together to create regulations that promote the responsible use of AI while also fostering innovation and economic growth. By addressing the challenges associated with regulating AI and taking into account the ethical implications of this technology, we can ensure that AI benefits society as a whole. It is imperative that we stay ahead of the curve and proactively address the challenges posed by AI through thoughtful and comprehensive legislation. 

In the ever-evolving landscape of AI technology, legislation will play a critical role in shaping the future of society. It is up to us to ensure that AI is harnessed for the greater good and that regulations are in place to protect individuals and communities from potential harm. As we navigate the complexities of AI legislation, let us strive to strike a balance between innovation and ethical considerations, setting the stage for a future where AI technology can truly benefit society.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).

In [ ]:
pip install gradio

In [ ]:
import gradio as gr
from crewai import Agent, Task, Crew

# Define Agents
researcher = Agent(
    role="Senior Research Specialist",
    goal="Uncover intricate insights into the given topic",
    backstory="A seasoned expert in extracting valuable information and providing detailed analysis on any given subject.",
    allow_delegation=False
)

writer = Agent(
    role="Content Writer",
    goal="Craft engaging and informative articles",
    backstory="An experienced content writer who can turn complex research into easy-to-understand and engaging articles.",
    allow_delegation=False
)

editor = Agent(
    role="Content Editor",
    goal="Ensure articles are polished, coherent, and engaging",
    backstory="An expert editor with a sharp eye for clarity, grammar, and flow, capable of turning drafts into publication-ready articles.",
    allow_delegation=False
)

# Function to setup tasks and run CrewAI
def run_crewai_article_writer(topic):
    # Define tasks
    research_task = Task(
        description=f"Conduct in-depth research on the topic: {topic} and provide a structured summary.",
        expected_output="A structured and detailed research summary covering important aspects, recent developments, and insights related to the topic.",
        agent=researcher
    )

    writing_task = Task(
        description=f"Using the research, write a full-length article on the topic: {topic} in 1000 words give various sub section",
        expected_output="An engaging, detailed, informative, and coherent article suitable for online publication.",
        agent=writer
    )

    editing_task = Task(
        description=f"Review and edit the article on {topic} for clarity, grammar, flow, and engagement.",
        expected_output="A polished, clear, and professionally edited final version of the article.",
        agent=editor
    )

    # Setup Crew
    crew = Crew(
        agents=[researcher, writer, editor],
        tasks=[research_task, writing_task, editing_task],
        verbose=True  # Set to False if you want less console output
    )

    # Execute
    result = crew.kickoff()
    return result

# Gradio Interface
def generate_article(topic):
    return run_crewai_article_writer(topic)

iface = gr.Interface(
    fn=generate_article,
    inputs=gr.Textbox(lines=2, placeholder="Enter a topic for the article..."),
    outputs=gr.Textbox(lines=20, label="Generated Article"),
    title="DDS CrewAI Article Writer with Researcher, Writer, and Editor",
    description="Enter a topic and let CrewAI agents research, write, and edit a complete article!"
)

if __name__ == "__main__":
    iface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://513f4ff8d873b9a3a0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
